<a href="https://colab.research.google.com/github/duynht/private-ai/blob/master/Section%201%20-%20Differential%20Privacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lesson: Toy Differential Privacy - Simple Database Queries

In this section we're going to play around with Differential Privacy in the context of a database query. The database is going to be a VERY simple database with only one boolean column. Each row corresponds to a person. Each value corresponds to whether or not that person has a certain private attribute (such as whether they have a certain disease, or whether they are above/below a certain age). We are then going to learn how to know whether a database query over such a small database is differentially private or not - and more importantly - what techniques are at our disposal to ensure various levels of privacy


### First We Create a Simple Database

Step one is to create our database - we're going to do this by initializing a random list of 1s and 0s (which are the entries in our database). Note - the number of entries directly corresponds to the number of people in our database.

In [ ]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([ True,  True,  True,  ..., False, False, False])

## Project: Generate Parallel Databases

Key to the definition of differenital privacy is the ability to ask the question "When querying a database, if I removed someone from the database, would the output of the query be any different?". Thus, in order to check this, we must construct what we term "parallel databases" which are simply databases with one entry removed. 

In this first project, I want you to create a list of every parallel database to the one currently contained in the "db" variable. Then, I want you to create a function which both:

- creates the initial database (db)
- creates all parallel databases

In [ ]:
db = torch.rand(num_entries) > 0.5
db

tensor([ True, False, False,  ..., False,  True, False])

In [ ]:
db[0:5]

tensor([ True, False, False,  True,  True])

In [ ]:
remove_index = 2

In [ ]:
def get_parallel_db(db, remove_index):
    
    return torch.cat((db[0:remove_index],db[remove_index + 1:]))

In [ ]:
get_parallel_db(db, remove_index).shape

torch.Size([4999])

In [ ]:
def get_parallel_dbs(db):
    
    parallel_dbs = list()
    
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
        
    return parallel_dbs

In [ ]:
pdbs = get_parallel_dbs(db)

In [ ]:
def create_db_and_parallels(num_entries):
    
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [ ]:
db, pdbs = create_db_and_parallels(20)

In [ ]:
db

tensor([ True,  True,  True, False, False, False,  True, False,  True,  True,
         True,  True, False,  True, False,  True, False, False, False, False])

In [ ]:
pdbs

[tensor([ True,  True, False, False, False,  True, False,  True,  True,  True,
          True, False,  True, False,  True, False, False, False, False]),
 tensor([ True,  True, False, False, False,  True, False,  True,  True,  True,
          True, False,  True, False,  True, False, False, False, False]),
 tensor([ True,  True, False, False, False,  True, False,  True,  True,  True,
          True, False,  True, False,  True, False, False, False, False]),
 tensor([ True,  True,  True, False, False,  True, False,  True,  True,  True,
          True, False,  True, False,  True, False, False, False, False]),
 tensor([ True,  True,  True, False, False,  True, False,  True,  True,  True,
          True, False,  True, False,  True, False, False, False, False]),
 tensor([ True,  True,  True, False, False,  True, False,  True,  True,  True,
          True, False,  True, False,  True, False, False, False, False]),
 tensor([ True,  True,  True, False, False, False, False,  True,  True,  True,
   

In [ ]:
db.shape

torch.Size([20])

# Lesson: Towards Evaluating The Differential Privacy of a Function

Intuitively, we want to be able to query our database and evaluate whether or not the result of the query is leaking "private" information. As mentioned previously, this is about evaluating **whether the output of a query changes when we remove someone from the database**. Specifically, we want to evaluate the *maximum* amount the query changes when someone is removed (maximum over all possible people who could be removed). So, in order to evaluate how much privacy is leaked, we're going to iterate over each person in the database and measure the difference in the output of the query relative to when we query the entire database. 

Just for the sake of argument, let's make our first "database query" a simple sum. Aka, we're going to count the number of 1s in the database.

In [ ]:
db, pdbs = create_db_and_parallels(5000)

In [ ]:
def query(db):
    return db.sum()

In [ ]:
full_db_result = query(db)

In [ ]:
sensitivity = 0
for pdb in pdbs:
    pdb_result = query(pdb)
    
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if(db_distance > sensitivity):
        sensitivity = db_distance

In [ ]:
sensitivity

tensor(1)

# Project - Evaluating the Privacy of a Function

In the last section, we measured the difference between each parallel db's query result and the query result for the entire database and then calculated the max value (which was 1). **This value is called "sensitivity"**, and it corresponds to the function we chose for the query. Namely, the "sum" query will always have a sensitivity of exactly 1. However, we can also calculate sensitivity for other functions as well.

Let's try to calculate sensitivity for the "mean" function.

In [ ]:
def sensitivity(query, n_entries=1000):
    
    db, pdbs = create_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    sensitivity = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > sensitivity):
            sensitivity = db_distance
    
    return sensitivity

In [ ]:
def query(db):
    return db.float().mean()

In [ ]:
sensitivity(query)

tensor(0.0005)

In [ ]:
db, pdbs = create_db_and_parallels(20)

In [ ]:
db

tensor([False,  True,  True, False,  True, False, False,  True,  True,  True,
        False, False,  True, False,  True,  True, False, False, False, False])

*Notes:* The notion of sensitivity is used to evaluate if the result of a query is truly the aggregation of information rather than **using** actual information of individuals

Wow! That sensitivity is WAY lower. Note the intuition here. "Sensitivity" is measuring how sensitive the output of the query is to a person being removed from the database. For a simple sum, this is always 1, but for the mean, removing a person is going to change the result of the query by rougly 1 divided by the size of the database (which is much smaller). Thus, "mean" is a VASTLY less "sensitive" function (query) than SUM.

# Project: Calculate L1 Sensitivity For Threshold

In this first project, I want you to calculate the sensitivty for the "threshold" function. 

- First compute the sum over the database (i.e. sum(db)) and return whether that sum is greater than a certain threshold.
- Then, I want you to create databases of size 10 and threshold of 5 and calculate the sensitivity of the function. 
- Finally, re-initialize the database 10 times and calculate the sensitivity each time.

In [ ]:
def query(db, threshold=5):    
    return (db.sum() > threshold).float()

In [ ]:
for i in range(10):    
    sens_f = sensitivity(query, n_entries=10)
    print(sens_f)

tensor(1.)
0
0
0
tensor(1.)
0
0
tensor(1.)
0
tensor(1.)


# Lesson: A Basic Differencing Attack

Sadly none of the functions we've looked at so far are differentially private (despite them having varying levels of sensitivity). The most basic type of attack can be done as follows.

Let's say we wanted to figure out a specific person's value in the database. All we would have to do is query for the sum of the entire database and then the sum of the entire database without that person!

# Project: Perform a Differencing Attack on Row 10

In this project, I want you to construct a database and then demonstrate how you can use two different sum queries to explose the value of the person represented by row 10 in the database (note, you'll need to use a database with at least 10 rows)

In [ ]:
db, _ = create_db_and_parallels(100)

In [ ]:
db

tensor([False,  True, False, False,  True,  True, False,  True,  True,  True,
        False,  True,  True, False,  True, False,  True,  True,  True, False,
         True,  True,  True, False, False, False, False,  True, False,  True,
        False, False, False, False, False,  True,  True,  True,  True,  True,
         True, False, False,  True, False, False,  True,  True,  True,  True,
         True, False,  True,  True,  True, False, False,  True,  True,  True,
        False,  True, False,  True, False,  True, False, False,  True,  True,
        False, False, False,  True, False,  True,  True,  True,  True,  True,
         True,  True,  True, False,  True,  True,  True,  True,  True,  True,
         True, False, False,  True, False, False, False, False, False,  True])

In [ ]:
pdb = get_parallel_db(db, remove_index=10)

In [ ]:
db[10]

tensor(False)

In [ ]:
# differecing attack using sum query
sum(db) - sum(pdb)

tensor(0)

In [ ]:
# differecing attack using mean query
(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(-0.0059)

In [ ]:
# differecing attack using threshold
(sum(db) > 49).float() - (sum(pdb) > 49).float()

tensor(0.)

# Project: Local Differential Privacy

As you can see, the basic sum query is not differentially private at all! In truth, differential privacy always requires a form of randomness added to the query. Let me show you what I mean.

### Randomized Response (Local Differential Privacy)

Let's say I have a group of people I wish to survey about a very taboo behavior which I think they will lie about (say, I want to know if they have ever committed a certain kind of crime). I'm not a policeman, I'm just trying to collect statistics to understand the higher level trend in society. So, how do we do this? One technique is to add randomness to each person's response by giving each person the following instructions (assuming I'm asking a simple yes/no question):

- Flip a coin 2 times.
- If the first coin flip is heads, answer honestly
- If the first coin flip is tails, answer according to the second coin flip (heads for yes, tails for no)!

Thus, each person is now protected with "plausible deniability". If they answer "Yes" to the question "have you committed X crime?", then it might becasue they actually did, or it might be becasue they are answering according to a random coin flip. Each person has a high degree of protection. Furthermore, we can recover the underlying statistics with some accuracy, as the "true statistics" are simply averaged with a 50% probability. Thus, if we collect a bunch of samples and it turns out that 60% of people answer yes, then we know that the TRUE distribution is actually centered around 70%, because 70% averaged wtih 50% (a coin flip) is 60% which is the result we obtained. 

However, it should be noted that, especially when we only have a few samples, this comes at the cost of accuracy. This tradeoff exists across all of Differential Privacy. The greater the privacy protection (plausible deniability) the less accurate the results. 

Let's implement this local DP for our database before!

In [ ]:
db, pdbs = create_db_and_parallels(100)

In [ ]:
# actual values from people
db

tensor([ True, False, False, False,  True,  True,  True, False, False,  True,
        False,  True,  True,  True,  True,  True, False, False, False,  True,
        False, False, False, False,  True, False,  True,  True, False, False,
         True, False,  True,  True, False,  True, False, False,  True, False,
        False, False, False,  True, False, False,  True, False,  True, False,
        False, False,  True,  True, False,  True, False,  True,  True,  True,
        False,  True, False, False,  True, False,  True,  True, False,  True,
         True, False,  True,  True, False,  True, False,  True, False, False,
        False,  True, False,  True, False,  True,  True, False, False, False,
        False, False,  True, False, False, False,  True, False, False,  True])

In [ ]:
def query(db):
    
    true_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    
    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
    
    db_result = torch.mean(augmented_database.float()) * 2 - 0.5
    
    return db_result, true_result

In [ ]:
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.7000)
Without noise:tensor(0.6000)


In [ ]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.5200)
Without noise:tensor(0.4900)


In [ ]:
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.5060)
Without noise:tensor(0.4890)


In [ ]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.5060)
Without noise:tensor(0.5001)


# Project: Varying Amounts of Noise

In this project, I want you to augment the randomized response query (the one we just wrote) to allow for varying amounts of randomness to be added. Specifically, I want you to bias the coin flip to be higher or lower and then run the same experiment. 

Note - this one is a bit tricker than you might expect. You need to both adjust the likelihood of the first coin flip AND the de-skewing at the end (where we create the "augmented_result" variable).

In [ ]:
def query(db, noise=0.2):

    true_result = torch.mean(db.float())

    first_coin_flip = (torch.rand(len(db)) > noise).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()

    augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
    
#     true_dist_mean = 0.7 # 70% of people said YES
#     noise_dist_mean = 0.5 # 50/50 coin flip
#     augmented_database_mean = (true_dist_mean * noise) + (noise_dist_mean * (1 - noise))

    sk_result = augmented_database.float().mean()

    private_result = ((sk_result / noise) - 0.5) * noise / (1 - noise)

    return private_result, true_result

In [ ]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.1)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.5444)
Without noise:tensor(0.5100)


In [ ]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.2)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.5375)
Without noise:tensor(0.5200)


In [ ]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.4)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.4667)
Without noise:tensor(0.5400)


In [ ]:
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db, noise=0.4)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.5017)
Without noise:tensor(0.5280)


In [ ]:
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db, noise=0.8)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.4750)
Without noise:tensor(0.4930)


In [ ]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db, noise=0.2)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.4976)
Without noise:tensor(0.4966)


In [ ]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db, noise=0.4)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.5045)
Without noise:tensor(0.5005)


In [ ]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db, noise=0.8)
print("With noise:" + str(private_result))
print("Without noise:" + str(true_result))

With noise:tensor(0.4945)
Without noise:tensor(0.4994)


*Notes:* The more *private* data you have access to, the easier it is to preserve the privacy

*Notes:* Differential Privacy is a complex filter that looks for information consistent across different data points, hence it is impossible to leak **unique** information.

# Lesson: The Formal Definition of Differential Privacy

The previous method of adding noise was called "Local Differentail Privacy" because we added noise to each datapoint individually. This is necessary for some situations wherein the data is SO sensitive that individuals do not trust noise to be added later. However, it comes at a very high cost in terms of accuracy. 

However, alternatively we can add noise AFTER data has been aggregated by a function. This kind of noise can allow for similar levels of protection with a lower affect on accuracy. However, participants must be able to trust that no-one looked at their datapoints _before_ the aggregation took place. In some situations this works out well, in others (such as an individual hand-surveying a group of people), this is less realistic.

Nevertheless, global differential privacy is incredibly important because it allows us to perform differential privacy on smaller groups of individuals with lower amounts of noise. Let's revisit our sum functions.

In [ ]:
db, pdbs = create_db_and_parallels(100)

def query(db):
    return torch.sum(db.float())

def M(db):
    query(db) + noise

query(db)

tensor(40.)

So the idea here is that we want to add noise to the output of our function. We actually have two different kinds of noise we can add - Laplacian Noise or Gaussian Noise. However, before we do so at this point we need to dive into the formal definition of Differential Privacy.

![alt text](dp_formula.png "Title")

_Image From: "The Algorithmic Foundations of Differential Privacy" - Cynthia Dwork and Aaron Roth - https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf_

This definition does not _create_ differential privacy, instead it is a measure of how much privacy is afforded by a query M. Specifically, it's a comparison between running the query M on a database (x) and a parallel database (y). As you remember, parallel databases are defined to be the same as a full database (x) with one entry/person removed.

Thus, this definition says that FOR ALL parallel databases, the maximum distance between a query on database (x) and the same query on database (y) will be e^epsilon, but that occasionally this constraint won't hold with probability delta. Thus, this theorem is called "epsilon delta" differential privacy.

# Epsilon

Let's unpack the intuition of this for a moment. 

Epsilon Zero: If a query satisfied this inequality where epsilon was set to 0, then that would mean that the query for all parallel databases outputed the exact same value as the full database. As you may remember, when we calculated the "threshold" function, often the Sensitivity was 0. In that case, the epsilon also happened to be zero.

Epsilon One: If a query satisfied this inequality with epsilon 1, then the maximum distance between all queries would be 1 - or more precisely - the maximum distance between the two random distributions M(x) and M(y) is 1 (because all these queries have some amount of randomness in them, just like we observed in the last section).

# Delta

Delta is basically the probability that epsilon breaks. Namely, sometimes the epsilon is different for some queries than it is for others. For example, you may remember when we were calculating the sensitivity of threshold, most of the time sensitivity was 0 but sometimes it was 1. Thus, we could calculate this as "epsilon zero but non-zero delta" which would say that epsilon is perfect except for some probability of the time when it's arbitrarily higher. Note that this expression doesn't represent the full tradeoff between epsilon and delta.

# Lesson: How To Add Noise for Global Differential Privacy

In this lesson, we're going to learn about how to take a query and add varying amounts of noise so that it satisfies a certain degree of differential privacy. In particular, we're going to leave behind the Local Differential privacy previously discussed and instead opt to focus on Global differential privacy. 

So, to sum up, this lesson is about adding noise to the output of our query so that it satisfies a certain epsilon-delta differential privacy threshold.

There are two kinds of noise we can add - Gaussian Noise or Laplacian Noise. Generally speaking Laplacian is better, but both are still valid. Now to the hard question...

### How much noise should we add?

The amount of noise necessary to add to the output of a query is a function of four things:

- the type of noise (Gaussian/Laplacian)
- the sensitivity of the query/function
- the desired epsilon (ε)
- the desired delta (δ)

Thus, for each type of noise we're adding, we have different way of calculating how much to add as a function of sensitivity, epsilon, and delta. We're going to focus on Laplacian noise. Laplacian noise is increased/decreased according to a "scale" parameter b. We choose "b" based on the following formula.

b = sensitivity(query) / epsilon

In other words, if we set b to be this value, then we know that we will have a privacy leakage of <= epsilon. Furthermore, the nice thing about Laplace is that it guarantees this with delta == 0. There are some tunings where we can have very low epsilon where delta is non-zero, but we'll ignore them for now.

### Querying Repeatedly

- if we query the database multiple times - we can simply add the epsilons (Even if we change the amount of noise and their epsilons are not the same).

# Project: Create a Differentially Private Query

In this project, I want you to take what you learned in the previous lesson and create a query function which sums over the database and adds just the right amount of noise such that it satisfies an epsilon constraint. Write a query for both "sum" and for "mean". Ensure that you use the correct sensitivity measures for both.

In [ ]:
epsilon = 0.0001

In [ ]:
import numpy as np

In [ ]:
db, pdbs = create_db_and_parallels(100)

In [ ]:
def sum_query(db):
    return db.sum()

In [ ]:
def mean_query(db):
    return db.float().mean()

In [ ]:
def laplacian_mechanism(db, query, sensitivity):
    
    beta = sensitivity / epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    
    return query(db) + noise

In [ ]:
laplacian_mechanism(db, sum_query, 1)

tensor([-25929.1468], dtype=torch.float64)

In [ ]:
laplacian_mechanism(db, mean_query, 1/100)

tensor([129.9379], dtype=torch.float64)

# Lesson: Differential Privacy for Deep Learning

So in the last lessons you may have been wondering - what does all of this have to do with Deep Learning? Well, these same techniques we were just studying form the core primitives for how Differential Privacy provides guarantees in the context of Deep Learning. 

Previously, we defined perfect privacy as "a query to a database returns the same value even if we remove any person from the database", and used this intuition in the description of epsilon/delta. In the context of deep learning we have a similar standard.

Training a model on a dataset should return the same model even if we remove any person from the dataset.

Thus, we've replaced "querying a database" with "training a model on a dataset". In essence, the training process is a kind of query. However, one should note that this adds two points of complexity which database queries did not have:

    1. do we always know where "people" are referenced in the dataset?
    2. neural models rarely never train to the same output model, even on identical data

The answer to (1) is to treat each training example as a single, separate person. Strictly speaking, this is often overly zealous as some training examples have no relevance to people and others may have multiple/partial (consider an image with multiple people contained within it). Thus, localizing exactly where "people" are referenced, and thus how much your model would change if people were removed, is challenging.

The answer to (2) is also an open problem - but several interesitng proposals have been made. We're going to focus on one of the most popular proposals, PATE.

## An Example Scenario: A Health Neural Network

First we're going to consider a scenario - you work for a hospital and you have a large collection of images about your patients. However, you don't know what's in them. You would like to use these images to develop a neural network which can automatically classify them, however since your images aren't labeled, they aren't sufficient to train a classifier. 

However, being a cunning strategist, you realize that you can reach out to 10 partner hospitals which DO have annotated data. It is your hope to train your new classifier on their datasets so that you can automatically label your own. While these hospitals are interested in helping, they have privacy concerns regarding information about their patients. Thus, you will use the following technique to train a classifier which protects the privacy of patients in the other hospitals.

- 1) You'll ask each of the 10 hospitals to train a model on their own datasets (All of which have the same kinds of labels)
- 2) You'll then use each of the 10 partner models to predict on your local dataset, generating 10 labels for each of your datapoints
- 3) Then, for each local data point (now with 10 labels), you will perform a DP query to generate the final true label. This query is a "max" function, where "max" is the most frequent label across the 10 labels. We will need to add laplacian noise to make this Differentially Private to a certain epsilon/delta constraint.
- 4) Finally, we will retrain a new model on our local dataset which now has labels. This will be our final "DP" model.

So, let's walk through these steps. I will assume you're already familiar with how to train/predict a deep neural network, so we'll skip steps 1 and 2 and work with example data. We'll focus instead on step 3, namely how to perform the DP query for each example using toy data.

So, let's say we have 10,000 training examples, and we've got 10 labels for each example (from our 10 "teacher models" which were trained directly on private data). Each label is chosen from a set of 10 possible labels (categories) for each image.

In [ ]:
import numpy as np

In [ ]:
num_teachers = 10 # we're working with 10 partner hospitals
num_examples = 10000 # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

In [ ]:
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int).transpose(1,0) # fake predictions

In [ ]:
preds[:,0]

array([4, 6, 5, ..., 4, 5, 2])

In [ ]:
new_labels = list()
for an_image in preds:

    label_counts = np.bincount(an_image, minlength=num_labels)

    epsilon = 0.1
    beta = 1 / epsilon

    for i in range(len(label_counts)):
        label_counts[i] += np.random.laplace(0, beta, 1)

    new_label = np.argmax(label_counts)
    
    new_labels.append(new_label)

In [ ]:
len(new_labels)

10000

# PATE Analysis

In [ ]:
labels = np.array([9, 9, 3, 6, 9, 9, 9, 9, 8, 2])
counts = np.bincount(labels, minlength=10)
query_result = np.argmax(counts)
query_result

9

In [ ]:
from syft.frameworks.torch.dp import pate

In [ ]:
num_teachers, num_examples, num_labels = (100, 100, 10)
preds = (np.random.rand(num_teachers, num_examples) * num_labels).astype(int) #fake preds
indices = (np.random.rand(num_examples) * num_labels).astype(int) # true answers

In [ ]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 11.756462732485105


In [ ]:
preds[:,0:5] *= 0

In [ ]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 7.867987172744542


In [ ]:
preds[:,0:50] *= 0

In [ ]:
data_dep_eps, data_ind_eps = pate.perform_analysis(teacher_preds=preds, indices=indices, noise_eps=0.1, delta=1e-5, moments=20)
print("Data Independent Epsilon:", data_ind_eps)
print("Data Dependent Epsilon:", data_dep_eps)

Data Independent Epsilon: 11.756462732485115
Data Dependent Epsilon: 0.9029013677789843


# Where to Go From Here


Read:
    - Algorithmic Foundations of Differential Privacy: https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf
    - Deep Learning with Differential Privacy: https://arxiv.org/pdf/1607.00133.pdf
    - The Ethical Algorithm: https://www.amazon.com/Ethical-Algorithm-Science-Socially-Design/dp/0190948205
   
Topics:
    - The Exponential Mechanism
    - The Moment's Accountant
    - Differentially Private Stochastic Gradient Descent

Advice:
    - For deployments - stick with public frameworks!
    - Join the Differential Privacy Community
    - Don't get ahead of yourself - DP is still in the early days

# Section Project:

For the final project for this section, you're going to train a DP model using this PATE method on the MNIST dataset, provided below.

## Preparing datasets

In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
std_transforms = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5,), (0.5,)),
                                    ])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=std_transforms)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=std_transforms)

In [ ]:
train_data = mnist_trainset.data
train_targets = mnist_trainset.targets

In [ ]:
import torch.utils.data as data

In [ ]:
num_teachers = 250
num_teacher_examples = 200
num_student_examples = len(mnist_trainset) - (num_teachers * num_teacher_examples)
teacher_datasets = [data.Subset(mnist_trainset, list(range(i * num_teacher_examples, (i+1) * num_teacher_examples))) for i in range(num_teachers)]
student_dataset = data.Subset(mnist_trainset, list(range(num_teacher_examples * num_teachers, len(mnist_trainset))))

In [ ]:
print(len(mnist_testset))
print(len(student_dataset))

10000
10000


## Import and set up classifiers

In [ ]:
# !pip install syft

In [ ]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

from syft.frameworks.torch.dp import pate

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
# Simple feedforward
def model():
  return nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64,10),
                      nn.LogSoftmax(dim=1))

## Training teacher models

In [ ]:
lr = 3e-2
num_epochs = 20
batch_size = 30

teacher_models = [model().to(device) for _ in range(num_teachers)]
teacher_loaders = [data.DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True) for dataset in teacher_datasets]
teacher_optimizers = [optim.Adam(classifier.parameters(), lr=lr) for classifier in teacher_models]

criterion = nn.NLLLoss()

for epoch in range(num_epochs):
  avg_losses = []
  avg_accuracies = []

  for index, classifier in enumerate(teacher_models):
    total_loss = 0
    total_acc = 0
    trainloader = teacher_loaders[index]
    optimizer = teacher_optimizers[index]

    for images, labels in trainloader:
      
      images = images.view(images.shape[0], -1)

      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      output = classifier(images)
      loss = criterion(output, labels)
      loss.backward()
      optimizer.step()

      total_loss += loss.item()
      total_acc += (output.exp().argmax(dim=1) == labels).sum().item()
    
    avg_losses.append(total_loss / len(trainloader))
    avg_accuracies.append(total_acc / (batch_size * len(trainloader)))

  print('Epoch {0}/{1}'.format(epoch + 1, num_epochs))
  print('avg_losses:{0}\n avg_accuracies: {1}'.format(avg_losses, avg_accuracies))

Epoch 1/20
avg_losses:[5.776464064915975, 9.790359457333883, 6.613523284594218, 12.658158421516418, 10.580695788065592, 9.426926016807556, 8.892938017845154, 10.478723605473837, 10.707446932792664, 9.137556155522665, 10.564786354700724, 12.618427991867065, 6.592108726501465, 8.377890268961588, 10.22396437327067, 9.201973875363668, 12.091495315233866, 8.595958471298218, 6.337315678596497, 10.902777830759684, 8.32911197344462, 10.12644350528717, 7.0481336911519366, 6.781358043352763, 9.204182147979736, 8.758175571759542, 10.21658464272817, 9.71798320611318, 7.802382667859395, 9.543087323506674, 8.825368563334147, 10.469084898630777, 9.835882107416788, 10.04987633228302, 9.470319231351217, 7.936081965764363, 10.646167437235514, 8.74842389424642, 8.118987321853638, 9.651824076970419, 9.19758145014445, 10.32101857662201, 6.688697298367818, 11.084139744440714, 6.7856234312057495, 8.027595440546671, 8.300583402315775, 7.319328387578328, 8.939846793810526, 6.8282701174418134, 11.77317957083384

## Evaluate teacher models

In [ ]:
batch_size = 1024
testloader = data.DataLoader(mnist_testset, batch_size=batch_size, shuffle=False, drop_last=True)

preds_mat = []
total_acc = 0
for images, labels in testloader:
  images = images.view(images.shape[0], -1)

  images = images.to(device)
  labels = labels.to(device)

  preds_list = []
  with torch.no_grad():
    for classifier in teacher_models:
      classifier.eval()

      output = classifier(images)
      preds_list.append(output.exp().argmax(dim=1).cpu())

      total_acc += (output.exp().argmax(dim=1) == labels).sum().item()
    
  preds_mat.append(preds_list)      

print("Average accuracy: ", total_acc / (num_teachers * batch_size * len(testloader)))


Average accuracy:  0.5439587673611112


In [ ]:
print(len(preds_mat))
print(num_teachers)

9
250


In [ ]:
import numpy as np

preds_tensor = torch.cat([torch.stack(preds_list, dim=0) for preds_list in preds_mat], dim=1)
preds_counts = torch.tensor(np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=0, arr=preds_tensor.numpy()))

aggregate_preds = preds_counts.argmax(dim=0)
labels = torch.cat([labels for _, labels in testloader], dim=0)

aggregate_acc = (aggregate_preds == labels).float().mean().item()

print("Aggregate accuracy: ", aggregate_acc)

Aggregate accuracy:  0.8728298544883728


## Aggreate teacher models

In [ ]:
import torch.distributions as dists

In [ ]:
def aggregate_counts(images):

  preds_list = []
  with torch.no_grad():
    for model in teacher_models:
      model.eval()
      preds_list.append(model(images).exp().argmax(dim=1).cpu())

  preds_tensor = torch.stack(preds_list, dim=0)
  counts = torch.tensor(np.apply_along_axis(lambda x: np.bincount(x, minlength=10), axis=0, arr=preds_tensor.numpy()))

  return counts.transpose(0,1), preds_tensor

## Train the student model

In [ ]:
lr = 3e-2
epsilon = 0.05
num_epochs = 20

student = model()

laplacian_mechanism = dists.Laplace(torch.zeros([], dtype=torch.float), torch.tensor(1/epsilon, dtype=torch.float))

trainloader = data.DataLoader(student_dataset, batch_size=1024, shuffle=False, drop_last=True)
optimizer = optim.Adam(student.parameters(), lr=lr)
criterion = nn.NLLLoss()

teacher_preds = []
noisy_labels = []

student.to(device)
for epoch in range(num_epochs):
  total_loss = 0
  total_acc = 0
  for images, _ in trainloader:  
    images = images.view(images.shape[0], -1)  
    images = images.to(device)
    
    label_pred_counts, label_preds = aggregate_counts(images)
    teacher_preds.append(label_preds)
    labels = (label_pred_counts + laplacian_mechanism.sample(label_pred_counts.size())).argmax(dim=1)
    noisy_labels.append(labels)
    
    labels = labels.to(device)

    output = student(images)
    optimizer.zero_grad()
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
    total_acc += (output.exp().argmax(dim=1) == labels).sum().item()
    
  print('Epoch {0}/{1}'.format(epoch + 1, num_epochs))
  print('avg_losses:{0}\n avg_accuracies: {1}'.format(total_loss / len(trainloader), total_acc / (batch_size * len(trainloader))))

teacher_preds = torch.cat(teacher_preds, dim=1)
noisy_labels = torch.cat(noisy_labels)

Epoch 1/20
avg_losses:7.209403885735406
 avg_accuracies: 0.1545138888888889
Epoch 2/20
avg_losses:1.918307688501146
 avg_accuracies: 0.3576388888888889
Epoch 3/20
avg_losses:1.5063204632865057
 avg_accuracies: 0.5005425347222222
Epoch 4/20
avg_losses:1.2518766588634915
 avg_accuracies: 0.5458984375
Epoch 5/20
avg_losses:1.0999530752499898
 avg_accuracies: 0.6043836805555556
Epoch 6/20
avg_losses:1.0128992464807298
 avg_accuracies: 0.6228298611111112
Epoch 7/20
avg_losses:0.9395002921422323
 avg_accuracies: 0.6871744791666666
Epoch 8/20
avg_losses:0.8726597494549222
 avg_accuracies: 0.7059461805555556
Epoch 9/20
avg_losses:0.8611497150527107
 avg_accuracies: 0.7150607638888888
Epoch 10/20
avg_losses:0.8311778704325358
 avg_accuracies: 0.7296006944444444
Epoch 11/20
avg_losses:0.80228810177909
 avg_accuracies: 0.7428385416666666
Epoch 12/20
avg_losses:0.7575584252675375
 avg_accuracies: 0.7648654513888888
Epoch 13/20
avg_losses:0.7401336630185446
 avg_accuracies: 0.7692057291666666
Epoch

## Evaluate student model

In [ ]:
total_acc = 0
for images, labels in testloader:
  images = images.view(images.shape[0], -1)

  images = images.to(device)
  labels = labels.to(device)
  
  student.eval()
  with torch.no_grad():

    output = student(images)

    total_acc += (output.exp().argmax(dim=1) == labels).sum().item()

print("Average accuracy: ", total_acc / (batch_size * len(testloader)))


Average accuracy:  0.7387152777777778


## PATE Analysis

In [ ]:
from syft.frameworks.torch.dp import pate

In [ ]:
# Noisy Predictions as indices
data_dep_eps, data_indep_eps = pate.perform_analysis(teacher_preds.numpy(), noisy_labels.numpy(), epsilon, moments=18)
print("Data Independent Epsilon:", data_indep_eps)
print("Data Dependent Epsilon:  ", data_dep_eps)

Data Independent Epsilon: 1854.7129254649706
Data Dependent Epsilon:   1540.9990873128033


In [ ]:
# Predictions without Noise as indices
data_dep_eps, data_indep_eps = pate.perform_analysis(teacher_preds.numpy(), np.argmax(np.apply_along_axis(lambda x: np.bincount(x, None, 10), 0, teacher_preds.numpy()), 0), epsilon, moments=18)
print("Data Independent Epsilon:", data_indep_eps)
print("Data Dependent Epsilon:  ", data_dep_eps)

In [ ]:
# True Labels as indices
data_dep_eps, data_indep_eps = pate.perform_analysis(teacher_preds.numpy(), mnist_testset.true_targets[student_labeled_dataset.indices], epsilon, moments=18)
print("Data Independent Epsilon:", data_indep_eps)
print("Data Dependent Epsilon:  ", data_dep_eps)